# Variable Selection Methods

How do we choose what explanatory variables to include in our model? Do we include all possible explanatory variables or a subset of the variables? In this tutorial, we will explore variable selection techniques in regression modeling using the Boston dataset from the MASS package. 

## The dataset

For this tutorial we will be using the Boston housing dataset contained in the **MASS** package in R. The data was originally published by Harrison, D. and Rubinfeld, D.L. 'Hedonic prices and the demand for clean air', J. Environ. Economics & Management, vol.5, 81-102, 1978. The dataset contains information concerning housing in the area of Boston. It contains 506 observations of 14 variables.

In [2]:
#Install and load the required packages
library(MASS)



#Load the Boston dataset
data(Boston)

In [4]:
head(Boston)

crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,black,lstat,medv
0.00632,18,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
0.02731,0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
0.02729,0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
0.03237,0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
0.06905,0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2
0.02985,0,2.18,0,0.458,6.430,58.7,6.0622,3,222,18.7,394.12,5.21,28.7


In [5]:
summary(Boston)

      crim                zn             indus            chas        
 Min.   : 0.00632   Min.   :  0.00   Min.   : 0.46   Min.   :0.00000  
 1st Qu.: 0.08204   1st Qu.:  0.00   1st Qu.: 5.19   1st Qu.:0.00000  
 Median : 0.25651   Median :  0.00   Median : 9.69   Median :0.00000  
 Mean   : 3.61352   Mean   : 11.36   Mean   :11.14   Mean   :0.06917  
 3rd Qu.: 3.67708   3rd Qu.: 12.50   3rd Qu.:18.10   3rd Qu.:0.00000  
 Max.   :88.97620   Max.   :100.00   Max.   :27.74   Max.   :1.00000  
      nox               rm             age              dis        
 Min.   :0.3850   Min.   :3.561   Min.   :  2.90   Min.   : 1.130  
 1st Qu.:0.4490   1st Qu.:5.886   1st Qu.: 45.02   1st Qu.: 2.100  
 Median :0.5380   Median :6.208   Median : 77.50   Median : 3.207  
 Mean   :0.5547   Mean   :6.285   Mean   : 68.57   Mean   : 3.795  
 3rd Qu.:0.6240   3rd Qu.:6.623   3rd Qu.: 94.08   3rd Qu.: 5.188  
 Max.   :0.8710   Max.   :8.780   Max.   :100.00   Max.   :12.127  
      rad              tax 

In [6]:
str(Boston)

'data.frame':	506 obs. of  14 variables:
 $ crim   : num  0.00632 0.02731 0.02729 0.03237 0.06905 ...
 $ zn     : num  18 0 0 0 0 0 12.5 12.5 12.5 12.5 ...
 $ indus  : num  2.31 7.07 7.07 2.18 2.18 2.18 7.87 7.87 7.87 7.87 ...
 $ chas   : int  0 0 0 0 0 0 0 0 0 0 ...
 $ nox    : num  0.538 0.469 0.469 0.458 0.458 0.458 0.524 0.524 0.524 0.524 ...
 $ rm     : num  6.58 6.42 7.18 7 7.15 ...
 $ age    : num  65.2 78.9 61.1 45.8 54.2 58.7 66.6 96.1 100 85.9 ...
 $ dis    : num  4.09 4.97 4.97 6.06 6.06 ...
 $ rad    : int  1 2 2 3 3 3 5 5 5 5 ...
 $ tax    : num  296 242 242 222 222 222 311 311 311 311 ...
 $ ptratio: num  15.3 17.8 17.8 18.7 18.7 18.7 15.2 15.2 15.2 15.2 ...
 $ black  : num  397 397 393 395 397 ...
 $ lstat  : num  4.98 9.14 4.03 2.94 5.33 ...
 $ medv   : num  24 21.6 34.7 33.4 36.2 28.7 22.9 27.1 16.5 18.9 ...


To find out more about the variables included in the dataset run the following:

In [7]:
?Boston

## Common criteria for model selection

Before we get into the different strategies for variable selection we need to introduce some criteria used to measure the best fit for our model. You will then need to choose which criteria to use when carrying out the different strategies for variable selection.

### AIC and BIC

The Akaike's Information Criterion (AIC) and Bayesian Iformation Criterion (BIC) are 2 well-known methods for statistical model selection when using models fitted by maximum likelihood. I have provided the formula's for each criterion below.

**AIC** $= 2k - 2ln(L) $

**BIC** $= kln(n) - 2ln(L) $

where $k = $number of parameters, $n = $ number of data points and $L = $ the likelihood.  

The aim when choosing the best model is to minimise the AIC or BIC for the fitted models. AIC tends to favor models with more parameters as it has a lower penalty for complexity. BIC tends to favor simpler models, especially as the sample size increases, due to the logarithmic penalty term. We can calculate the AIC and BIC for fitted models using the AIC() and BIC() functions in R, which we will do in the following sections. For more information about __[AIC](https://en.wikipedia.org/wiki/Akaike_information_criterion)__ and __[BIC](https://en.wikipedia.org/wiki/Bayesian_information_criterion)__ click these links.




## Forward selection algorithm

When using the forward selection algorithm we start with the simplest model then one by one we add the variable that reduces AIC or BIC of the model the most. This process is repeated until we cannot reduce the AIC or BIC by adding another variable. The response variable for our model is medv (median value of owner-occupied homes in $1000s.) and we want to explore which explanatory variables, if any, have a significant impact on our response variable.

In [3]:
predictors <- names(Boston) [-grep('medv', names(Boston))] #vector of explanatory variable names
formula <- as.formula(paste("y ~ ", paste(names(Boston[,predictors]), collapse = "+")))
model_fs <- lm(medv ~ 1, data = Boston)
selection_fs <- step(model_fs, direction="forward", trace = TRUE, k=log(nrow(Boston)), scope = list(upper=formula))


Start:  AIC=2250.74
medv ~ 1

          Df Sum of Sq   RSS    AIC
+ lstat    1   23243.9 19472 1859.5
+ rm       1   20654.4 22062 1922.6
+ ptratio  1   11014.3 31702 2106.1
+ indus    1    9995.2 32721 2122.1
+ tax      1    9377.3 33339 2131.6
+ nox      1    7800.1 34916 2154.9
+ crim     1    6440.8 36276 2174.3
+ rad      1    6221.1 36495 2177.3
+ age      1    6069.8 36647 2179.4
+ zn       1    5549.7 37167 2186.6
+ black    1    4749.9 37966 2197.3
+ dis      1    2668.2 40048 2224.3
+ chas     1    1312.1 41404 2241.2
<none>                 42716 2250.7

Step:  AIC=1859.46
medv ~ lstat

          Df Sum of Sq   RSS    AIC
+ rm       1    4033.1 15439 1748.3
+ ptratio  1    2670.1 16802 1791.1
+ chas     1     786.3 18686 1844.8
+ dis      1     772.4 18700 1845.2
+ age      1     304.3 19168 1857.7
+ tax      1     274.4 19198 1858.5
<none>                 19472 1859.5
+ black    1     198.3 19274 1860.5
+ zn       1     160.3 19312 1861.5
+ crim     1     146.9 19325 1861.9


Using k = log(nrow(Boston)) in the step function changes the measure used from AIC to BIC, even though it is still listed as AIC in the output. As you can see from the output, lstat was the first variable added to the model followed by rm, ptratio, dis, nox, chas, black and zn. The output also shows the AIC (or in this case BIC) of the model when each variable is added to the current model. Now we can view our final model using the summary() function.

In [4]:
summary(selection_fs)


Call:
lm(formula = medv ~ lstat + rm + ptratio + dis + nox + chas + 
    black + zn, data = Boston)

Residuals:
     Min       1Q   Median       3Q      Max 
-15.6996  -2.7925  -0.5477   1.7005  27.6510 

Coefficients:
              Estimate Std. Error t value Pr(>|t|)    
(Intercept)  30.316950   4.870856   6.224 1.03e-09 ***
lstat        -0.543125   0.047652 -11.398  < 2e-16 ***
rm            4.116082   0.408594  10.074  < 2e-16 ***
ptratio      -0.881851   0.115718  -7.621 1.29e-13 ***
dis          -1.382714   0.187604  -7.370 7.15e-13 ***
nox         -16.687428   3.228873  -5.168 3.43e-07 ***
chas          3.111062   0.870076   3.576 0.000384 ***
black         0.009404   0.002639   3.563 0.000401 ***
zn            0.037808   0.013298   2.843 0.004652 ** 
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 4.847 on 497 degrees of freedom
Multiple R-squared:  0.7266,	Adjusted R-squared:  0.7222 
F-statistic: 165.1 on 8 and 497 DF,  p-value: <

## Backward selection algorithm

The backward selction algorith works in the opposite fashion to the forward selection algorithm. We start with with a model containing all possible explanatory variables and one-by-one we subtract a the variable that minimises the AIC or BIC for the model. This process is repeated until subtracting a variable does not lower the criterion of choice of the model.

In [5]:
#Create full linear regression model
model <- lm(medv ~ ., data = Boston)

summary(model)


Call:
lm(formula = medv ~ ., data = Boston)

Residuals:
    Min      1Q  Median      3Q     Max 
-15.595  -2.730  -0.518   1.777  26.199 

Coefficients:
              Estimate Std. Error t value Pr(>|t|)    
(Intercept)  3.646e+01  5.103e+00   7.144 3.28e-12 ***
crim        -1.080e-01  3.286e-02  -3.287 0.001087 ** 
zn           4.642e-02  1.373e-02   3.382 0.000778 ***
indus        2.056e-02  6.150e-02   0.334 0.738288    
chas         2.687e+00  8.616e-01   3.118 0.001925 ** 
nox         -1.777e+01  3.820e+00  -4.651 4.25e-06 ***
rm           3.810e+00  4.179e-01   9.116  < 2e-16 ***
age          6.922e-04  1.321e-02   0.052 0.958229    
dis         -1.476e+00  1.995e-01  -7.398 6.01e-13 ***
rad          3.060e-01  6.635e-02   4.613 5.07e-06 ***
tax         -1.233e-02  3.760e-03  -3.280 0.001112 ** 
ptratio     -9.527e-01  1.308e-01  -7.283 1.31e-12 ***
black        9.312e-03  2.686e-03   3.467 0.000573 ***
lstat       -5.248e-01  5.072e-02 -10.347  < 2e-16 ***
---
Signif. codes:  0

[1] 3091.007

In [7]:
selection_bs <- step(model, direction="backward", trace = TRUE, k=log(nrow(Boston)))

Start:  AIC=1648.81
medv ~ crim + zn + indus + chas + nox + rm + age + dis + rad + 
    tax + ptratio + black + lstat

          Df Sum of Sq   RSS    AIC
- age      1      0.06 11079 1642.6
- indus    1      2.52 11081 1642.7
<none>                 11079 1648.8
- chas     1    218.97 11298 1652.5
- tax      1    242.26 11321 1653.5
- crim     1    243.22 11322 1653.6
- zn       1    257.49 11336 1654.2
- black    1    270.63 11349 1654.8
- rad      1    479.15 11558 1664.0
- nox      1    487.16 11566 1664.4
- ptratio  1   1194.23 12273 1694.4
- dis      1   1232.41 12311 1696.0
- rm       1   1871.32 12950 1721.6
- lstat    1   2410.84 13490 1742.2

Step:  AIC=1642.59
medv ~ crim + zn + indus + chas + nox + rm + dis + rad + tax + 
    ptratio + black + lstat

          Df Sum of Sq   RSS    AIC
- indus    1      2.52 11081 1636.5
<none>                 11079 1642.6
- chas     1    219.91 11299 1646.3
- tax      1    242.24 11321 1647.3
- crim     1    243.20 11322 1647.3
- zn       1

[1] 3023.726

Using the backward selection algorithm step 1 subtracts age from the model and then step 2 subtracts the indus variable from the model. After step 2 we cannot improve our model further any further by subtracting a variable. The final model can be viewed using the summary() function.

In [8]:
summary(selection_bs)


Call:
lm(formula = medv ~ crim + zn + chas + nox + rm + dis + rad + 
    tax + ptratio + black + lstat, data = Boston)

Residuals:
     Min       1Q   Median       3Q      Max 
-15.5984  -2.7386  -0.5046   1.7273  26.2373 

Coefficients:
              Estimate Std. Error t value Pr(>|t|)    
(Intercept)  36.341145   5.067492   7.171 2.73e-12 ***
crim         -0.108413   0.032779  -3.307 0.001010 ** 
zn            0.045845   0.013523   3.390 0.000754 ***
chas          2.718716   0.854240   3.183 0.001551 ** 
nox         -17.376023   3.535243  -4.915 1.21e-06 ***
rm            3.801579   0.406316   9.356  < 2e-16 ***
dis          -1.492711   0.185731  -8.037 6.84e-15 ***
rad           0.299608   0.063402   4.726 3.00e-06 ***
tax          -0.011778   0.003372  -3.493 0.000521 ***
ptratio      -0.946525   0.129066  -7.334 9.24e-13 ***
black         0.009291   0.002674   3.475 0.000557 ***
lstat        -0.522553   0.047424 -11.019  < 2e-16 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*

You may have noticed that the forward and backward algorithm have given 2 different models. This is not always the case. Moving forward you could test both the models performance using cross validation or prediction methods. If we would like to find the model with the lowest AIC/BIC from both backward and forward selction algorithms we can change the direction argument in the step() function to direction = "both" which in this case would return the model selected through backward selection.

In [11]:
selection_bs <- step(model, direction="both", trace = TRUE, k=log(nrow(Boston)))

Start:  AIC=1648.81
medv ~ crim + zn + indus + chas + nox + rm + age + dis + rad + 
    tax + ptratio + black + lstat

          Df Sum of Sq   RSS    AIC
- age      1      0.06 11079 1642.6
- indus    1      2.52 11081 1642.7
<none>                 11079 1648.8
- chas     1    218.97 11298 1652.5
- tax      1    242.26 11321 1653.5
- crim     1    243.22 11322 1653.6
- zn       1    257.49 11336 1654.2
- black    1    270.63 11349 1654.8
- rad      1    479.15 11558 1664.0
- nox      1    487.16 11566 1664.4
- ptratio  1   1194.23 12273 1694.4
- dis      1   1232.41 12311 1696.0
- rm       1   1871.32 12950 1721.6
- lstat    1   2410.84 13490 1742.2

Step:  AIC=1642.59
medv ~ crim + zn + indus + chas + nox + rm + dis + rad + tax + 
    ptratio + black + lstat

          Df Sum of Sq   RSS    AIC
- indus    1      2.52 11081 1636.5
<none>                 11079 1642.6
- chas     1    219.91 11299 1646.3
- tax      1    242.24 11321 1647.3
- crim     1    243.20 11322 1647.3
- zn       1

## Leaps and Bounds

Models selected using the backward and forward algorithms result in nested models. Another method we could use is taking the best subset of explanatory variables that minimise some criterion of importance. This is where the leaps() function from the **leaps** package comes in. leaps() performs an exhaustive search for the best subsets of the variables in x for predicting y in linear regression, using an efficient branch-and-bound algorithm. To measure best model fit we can choose from __[Mallows' Cp](https://en.wikipedia.org/wiki/Mallows%27s_Cp)__, $R^2$ or adjusted $R^2$. We will be using Mallows' $C_p$ in the following code.

In [12]:
#Install and load "leaps" package
install.packages("leaps")
library(leaps)

y <- Boston$medv #Response Variable
x <- Boston[,1:13] #Explanatory variable matrix

selection_leaps <- leaps(x, y, method = "Cp", nbest = 1, strictly.compatible = FALSE)



Warning message:
"unable to access index for repository https://cran.r-project.org/bin/windows/contrib/3.6:
  cannot open URL 'https://cran.r-project.org/bin/windows/contrib/3.6/PACKAGES'"Package which is only available in source form, and may need
  compilation of C/C++/Fortran: 'leaps'


  These will not be installed


ERROR: Error in library(leaps): there is no package called 'leaps'


To visualise the $C_p$ value for the different models you can use the following code.

In [ ]:
plot(selection_leaps$size-1, selection_leaps$Cp, xlab = "Number of predictors", ylab = "Cp")

The "best model" is the one that minimises Mallows' $C_p$. In this case it is a model using 11 out of the possible 13 explanatory variables.

In [ ]:
best.model <- selection_leaps$which[which((selection_leaps$Cp == min(selection_leaps$Cp))),]

formula <- as.formula(paste("y ~ ", paste(colnames(x)[best.model], collapse = "+")))

model_leaps <- lm(formula, data = Boston)
summary(model_leaps)

It turns out that the best model found using the leaps() method is the same model that was chosen using the backward selection algorithm.